In [6]:
import pandas as pd 
import os

def save_to_csv(df, dataframe_name):
    filepath = f"../staging_2/{dataframe_name}/{dataframe_name}.csv"
    directory = os.path.dirname(filepath)
    if not os.path.exists(directory):
        os.makedirs(directory)
    df.to_csv(filepath, index=False)

In [7]:

# ADDING CALCULATED COLUMN list_price_egp
order_items=pd.read_csv("../staging_1/order_items/order_items.csv")
Exchange=pd.read_csv("../staging_1/Exchange/Exchange.csv")

order_items['key'] = 1
Exchange['key'] = 1

# Perform cross join
merged_df = pd.merge(order_items, Exchange, on='key').drop('key', axis=1)

# Calculate list_price_egp
order_items['list_price_egp'] = merged_df['list_price'] * merged_df['EGP']




In [8]:

# ADDING 3 CALCULATED COLUMNS IN ORDERS 

orders=pd.read_csv("../staging_1/orders/orders.csv")

orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['required_date'] = pd.to_datetime(orders['required_date'])
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'])

orders['late_delivery'] = orders['shipped_date'] > orders['required_date']

orders['latency_days'] = (orders['shipped_date'] - orders['required_date']).dt.days

orders['weekend'] = orders['shipped_date'].dt.dayofweek.isin([6, 0])  # Sunday or Monday

# LOOKUP TABLE 

status_lookup = {
    1: 'Pending',
    2: 'Processing',
    3: 'Rejected',
    4: 'Completed'
}
# Create DataFrame for order status lookup
status_df = pd.DataFrame(list(status_lookup.items()), columns=['order_status', 'status_description'])

# Merge orders DataFrame with status lookup DataFrame
orders = pd.merge(orders, status_df, on='order_status', how='inner')



# selected_orders = orders[orders['latency_days'] > 0]

# selected_orders 

In [9]:

# ADDING LOCAL FLAG FOR CUTOMERS

customers = pd.read_csv("../staging_1/customers/customers.csv")
stores = pd.read_csv("../staging_1/stores/stores.csv")
store_cities = stores['city'].unique()
customers['local'] = customers['city'].isin(store_cities)




In [10]:
brands=pd.read_csv("../staging_1/brands/brands.csv")
categories=pd.read_csv("../staging_1/categories/categories.csv")
Exchange=pd.read_csv("../staging_1/Exchange/Exchange.csv")
products=pd.read_csv("../staging_1/products/products.csv")
staffs=pd.read_csv("../staging_1/staffs/staffs.csv")
stocks=pd.read_csv("../staging_1/stocks/stocks.csv")
stores=pd.read_csv("../staging_1/stores/stores.csv")


save_to_csv(order_items,"order_items")
save_to_csv(orders,"orders")
save_to_csv(customers,"customers")
save_to_csv(brands,"brands")
save_to_csv(categories,"categories")
save_to_csv(customers,"customers")
save_to_csv(Exchange,"Exchange")
save_to_csv(products,"products")
save_to_csv(staffs,"staffs")
save_to_csv(stores,"stores")